In [1]:
import numpy as np
#from scipy import stats
from scipy.odr import *
%matplotlib inline
from matplotlib import pyplot as plt

In [2]:
import sys
sys.path.append('../lib')
from simlib import *

In [3]:
method = 'lang'
num_particles = 10000

In [4]:
def linear_func(B, x):
    return B[0]*x+B[1]

In [5]:
dt = 0.1
max_t = 10
steps = int(max_t/dt)

In [6]:
N = 10
Ds = np.linspace(10, 0.5, N)
ms = np.zeros(N)
sd = np.zeros(N)

for d, D in enumerate(Ds):
    ts, xs = simulate(zero_potential(), method=method, num_particles=num_particles, D=D, max_t=max_t, dt=dt)
    # xs[:,i] is the trajectory of particle i
    SD = np.zeros(shape=(num_particles, steps-1))
    for i in range(num_particles):
        SD[i] = [(xs[t,i] - xs[0,i])**2 for t in range(steps-1)]

    # SD[:,t] is an array with all the different SD values
    # (from the different particles) in the time step t
    MSD = np.zeros(steps-1)
    VAR = np.zeros(steps-1)
    for t in range(steps-1):
        MSD[t] = np.mean(SD[:,t])
        VAR[t] = np.var(SD[:,t])
    
    linear = Model(linear_func)
    data = RealData(ts[:-1], MSD, np.max(VAR))
    odr = ODR(data, linear, beta0=[1.3, 0.4])
    out = odr.run()
    ms[d] = out.beta[0]
    sd[d] = out.sd_beta[0]

100%|██████████| 99/99 [00:02<00:00, 37.27it/s]


In [8]:
for d, m in zip(ms, Ds):
    print(d/2, m)

10.0494214579 10.0
9.15675325835 8.94444444444
7.80558727222 7.88888888889
6.73670368404 6.83333333333
5.74912351199 5.77777777778
4.65686492506 4.72222222222
3.67946350452 3.66666666667
2.58101552717 2.61111111111
1.55580446776 1.55555555556
0.499566976654 0.5


In [10]:
data = RealData(Ds, ms, sy=sd)
odr = ODR(data, linear, beta0=[2, 0.0])
out = odr.run()
out.beta[0], out.sd_beta[0]

(2.0171789089030296, 0.018690436390571857)